### Introduction to Tray processing 
  by Andrii Terliuk


#### Simple introduction
First, lets run a couple of examples and understand how the tray works. The main idea of tray - it is an interface that passes events/frames from one module to another one. 

There is an example of running tray in `TrayProcessingExample.py` and `TrayFunctionModule.py`. The tray is created by running 

`tray = I3Tray()`
and then adding modules as 
```python
tray.AddModule(reader/generator)
tray.AddModule(SomeModule_1)
tray.AddModule(SomeModule_2)
...
tray.AddModule(writer,)
```

The tray is then executed at line `tray.Execute(n_frames)`, where `n_frames` is number of frames to run, if omitted - all frames will be processes/executed.

In [1]:
from icecube import dataclasses,dataio
import numpy as np

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<char, std::allocator<char> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, std::allocator<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > > > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<OMKey, std::allocator<OMKey> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<short, std::allocator<short> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<int, std::allocator<int> > already registered; second conversion method ignore

### Basic explanation of tray usage example:

First - we create tray
```python
tray = I3Tray()
```
Then we add an infinite stream of DAQ frames. It will be producing empty DAQ event frames:
```python
tray.AddModule("I3InfiniteSource","streams",
                Stream=icetray.I3Frame.DAQ,
               )
```
And now we add two simple example modules from `ExampleModules.py`. The first one is an example of generator module that in this case produces a vector of random doubles of given length.
```python
from ExampleModules import ExampleGenerator
tray.AddModule(ExampleGenerator, 
               Size=1000, 
               Mean=0.0, 
               Sigma = 10.0, 
               Outname ="RandomVector",
               Nevents = options.NEVENTS)
```
And the next module just calculates average  as module
```python
from ExampleModules import AveragingModule
tray.AddModule(ExampleModule, 
               Input="RandomVector",
               Output ="AverageFromModule"
               )
```
and as a function (same as module). However, we have to explicitly define which streams will be used (otherwise, only P-frames are processed with this function)
```python
from ExampleModules import AveragingFunction
tray.AddModule(AveragingFunction, 
               Input="RandomVector",
               Output ="AverageFromFunction", 
               Streams = [icetray.I3Frame.DAQ]
               )
```

and finally - we want to write the output
```
tray.AddModule("I3Writer","writer",
    Filename = options.OUTFILE,
    Streams = [icetray.I3Frame.DAQ],
    )
```
Once tray is created, we have only to execute it 
```python
tray.Execute()
```
#### Let's run the script now

In [11]:
! python3 IceTrayProcessingExample.py -o test_ex1.i3 -n 100

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<char, std::allocator<char> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, std::allocator<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > > > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<OMKey, std::allocator<OMKey> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<short, std::allocator<short> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<int, std::allocator<int> > already registered; second conversion method ignore

#### And let's check what is inside first frame

In [12]:
infile1= dataio.I3File("test_ex1.i3")
frame1= infile1.pop_daq()
print(frame1)

[ I3Frame  (DAQ):
  'AverageFromFunction' [DAQ] ==> I3PODHolder<double> (36)
  'AverageFromModule' [DAQ] ==> I3PODHolder<double> (36)
  'RandomVector' [DAQ] ==> I3Vector<double> (8038)
]



and inside created objects and print first to elements of created vector and computed average

In [13]:
vec = frame1['RandomVector']
print(type(vec), len(vec))
print(vec[0:10])
print("Average from module : ", frame1['AverageFromModule'].value)
print("Average from function : ", frame1['AverageFromFunction'].value)

<class 'icecube._dataclasses.I3VectorDouble'> 1000
[4.2794, -8.26218, -3.76731, 6.00036, 10.9956, -6.4164, -5.46859, -1.24741, -21.1018, -5.97079]
Average from module :  0.6624098840944088
Average from function :  0.6624098840944088


### Example of module class

A typical module class has the following strucuture
```python
class ExampleGenerator(icetray.I3Module):

    def __init__(self, context):
        icetray.I3Module.__init__(self, context)
        ...
  
    def Configure(self):
        ... 

    def Geometry(self, frame):
        ...
    def DAQ(self, frame):
        ...
    def Physics(self, frame):
        ...

```
The first initialization part adds parameters or creates parts of module that are "static".

In configuration, input parameters are read and configuration of the module is perfromed. 

At the end, `DAQ` funtion processes DAQ frames and pushes it further. The same structure functions exist for `Physics`, `Simulation`, `Geometry` etc frames. 



#### Processing module
Let's look inside the `ExampleModules.py` for examples of the modules. A simple module that caclulates average is explained below . First, we create a simple initialization that adds two options - input and output names: 
```python
class ExampleModule(icetray.I3Module):
    """
    Simple module that calculate average as an example 
    """
    def __init__(self, context):
        """
        This is initialization function, where one defines parameters
        """
        icetray.I3Module.__init__(self, context)
        self.AddParameter("Input", "input name", "")  
        self.AddParameter("Output", "output name", "")
        self.AddOutBox('OutBox')   
```
In configuration we read the values of these parameters
```python
    def Configure(self):
        """
        This function configures the module, for example - get parameters etc.
        """
        self.input = self.GetParameter("Input")
        self.output = self.GetParameter("Output")
```
And at the  end, this is the function that creates avearage of the vector, writes it to output and pushes the frame further 
```python
    def DAQ(self, frame):
        """
        Processing of the frame
        """
        frame[self.output] = dataclasses.I3Double(np.mean(frame[self.input]) )
        self.PushFrame(frame)
```

#### Example of generator module

This module puts information to the frame and, in this particular case, stops processing and interrupts the tray once it created certain number of frames. Its structure is rather similar to previous example, however, there is one small detail in configuration, where I add counter
```python
def Configure(self):
    ...
    self.counter = 0
```
and in frame processing, i increase number of counter. And once it reaches desired value - interrupt the process:
```python
    def DAQ(self, frame):
        ...
        self.PushFrame(frame)
        self.counter +=1
        if self.counter >= self.nevents:
            self.RequestSuspension()
```

#### Example of function module

Often, simple calculations can be done with a simple function and full module class is not necessary. This can particularly useful for easy funtions or during testing. In this example I crate function that expectes frame and has two parameters - `input` and `output`. Inside - it does very basic calculation and returns `True`. If `False` is return, then frame is dropped from further processing (and output)
```python
def AveragingFunction(frame, input, output):
    frame[output] = dataclasses.I3Double(np.array(frame[input]).mean())
    return(True)

```
When frame returns True, it is passed further. However, one can return False for certain frames. This process will remove frame from processing and will remove it. To add this functional module, ones need to do the following: 
```python
tray.AddModule(GetAverage, 
               input="RandomVector", 
               output= "AverageFromFunction",
               Streams = [icetray.I3Frame.DAQ])
```
It is important to note, that is function runs on `Physics` frames by default and to force it ti use DAQ ones, one has to add `Streams = [icetray.I3Frame.DAQ]`.

Let's run it and check the output

### Summary

This example showed basics of event processing with the tray. It gives user an ability to write and process events in extremely flexible way. In particular, one can store a large variety of hierarchical/heterogeneous data inside the frames. More complicated object corresponding to real physics processes can be created as well. Further information can be found in DataTypesExample notebook.